In [ ]:
%pip install openai ipynb

import os
import json
import random
from openai import OpenAI

api_key = ''
client = OpenAI(api_key=api_key)

In [14]:
# function that split the playlist into train and test set
def split_train_test(playlist, test_size=0.1):

    # random shuffle the playlist
    tracks = playlist['tracks']
    random.shuffle(tracks)

    num_tracks = len(tracks)
    num_test_tracks = int(num_tracks * test_size)

    test_set = tracks[:num_test_tracks]
    train_set = {
        "playlist_name": playlist["playlist_name"],
        "tracks": tracks[num_test_tracks:]
    }

    return train_set, test_set

In [13]:
# function that convert playlist dictionary to string
def to_str(playlist):
    playlist_str = f"Playlist Name: {playlist['playlist_name']}\n"
    playlist_str += "Tracks:\n"
    for track in playlist['tracks']:
        playlist_str += f"- {track['track_name']} by {track['artist_name']}\n"

    return playlist_str

In [4]:
# function that prompt the llm to generate response
def generate(system_prompt, user_input):
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            temperature=0, # using greedy decoding
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
          )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [6]:
# load dataset
input_file = 'cleaned_playlists.json'
with open(input_file, 'r') as inputFile:
    data = json.load(inputFile)

In [39]:
# prompts
feature_prompt = ''
recommendation_prompt = ''

In [ ]:
# main loop
for playlist in data['playlists']:
    train_set, test_set = split_train_test(playlist)
    train_set_str = to_str(train_set)
    features = generate(feature_prompt, train_set_str)
    user_input = train_set_str + "\n" + features
    recommendations = generate(recommendation_prompt, user_input)